In [ ]:
!pip install transformers

In [ ]:
!pip install sentence_transformers

In [1]:
import pandas as pd
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer
from sentence_transformers import SentenceTransformer, util

c:\Users\nasser\.conda\envs\finall\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv("merged_data.csv")

In [3]:
truth = data["Verse"]

In [4]:
truth = truth[:16]

In [5]:
truth

0             ah why this boding start this sudden pain
1      that wings my pulse and shoots from vein to vein
2             what mean regardless of yon midnight bell
3        these earthborn visions saddening o'er my cell
4     what strange disorder prompts these thoughts t...
5         these sighs to murmur and these tears to flow
6                  'tis she 'tis eloisa's form restor'd
7         once a pure saint and more than saints ador'd
8           she comes in all her killing charms confest
9       glares thro' the gloom and pours upon my breast
10     bids heav'n's bright guard from paraclete remove
11                 and drags me back to misery and love
12                 enjoy thy triumphs dear illusion see
13               this sad apostate from his god to thee
14             see at thy call my guilty warmths return
15        flame thro' my blood and steal me from my urn
Name: Verse, dtype: object

In [6]:
seeds = []
for line in truth:
  seeds.append(line.split(" ")[0]+" "+line.split(" ")[1])

In [7]:
seeds

['ah why',
 'that wings',
 'what mean',
 'these earthborn',
 'what strange',
 'these sighs',
 "'tis she",
 'once a',
 'she comes',
 "glares thro'",
 "bids heav'n's",
 'and drags',
 'enjoy thy',
 'this sad',
 'see at',
 "flame thro'"]

In [9]:
model = TFGPT2LMHeadModel.from_pretrained('gpt2_fine_tuned_model_english')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2_tokenizer_fine_tuned_model_english')

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2_fine_tuned_model_english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [10]:
predicted = []
for seed in seeds:

  seed_test= seed
  input_ids = tokenizer.encode(seed_test, return_tensors='tf')
  sample_outputs = model.generate(
    input_ids,
    do_sample=True,
    max_length=15,
    top_k=0,
    top_p=0.9,
    temperature=0.8,
    num_return_sequences=1
  )

  predicted.append(tokenizer.decode(sample_outputs[0], skip_special_tokens=True))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

In [11]:
predicted

['ah why to the distant crywhere as you are no more ye shall be',
 'that wings and shall shew brighti on me with such a lustre',
 'what mean to you to the brute and not to the brutelyfor the',
 "these earthborn god are'dwith thy glowing and lightning hand they shall",
 "what strange yet still the savage's madours have ye not yet wilt",
 'these sighs and pangs of grief and joyfellow of the common',
 "'tis she the true queen of truth and a poetess with a thousand",
 'once a train in time to riseand safe for ever once will be the',
 'she comes from abroad in camels or goats to teach thee the ways of',
 "glares thro' all the joys of lifeand now weep'd the",
 "bids heav'n's softest motionhis roaring oars may calm",
 "and drags me on the dusty road'ring mountain paths and amid the",
 "enjoy thy fruits the bestow'dof the birds and the trees for",
 "this sad fate thro' my heart to that just landwhere the pain which",
 'see at last the caprice of dutyfor the sole cause of love and',
 "flame thr

In [12]:
truth

0             ah why this boding start this sudden pain
1      that wings my pulse and shoots from vein to vein
2             what mean regardless of yon midnight bell
3        these earthborn visions saddening o'er my cell
4     what strange disorder prompts these thoughts t...
5         these sighs to murmur and these tears to flow
6                  'tis she 'tis eloisa's form restor'd
7         once a pure saint and more than saints ador'd
8           she comes in all her killing charms confest
9       glares thro' the gloom and pours upon my breast
10     bids heav'n's bright guard from paraclete remove
11                 and drags me back to misery and love
12                 enjoy thy triumphs dear illusion see
13               this sad apostate from his god to thee
14             see at thy call my guilty warmths return
15        flame thro' my blood and steal me from my urn
Name: Verse, dtype: object

In [13]:
# print("Output:", tokenizer.decode(sample_outputs[0], skip_special_tokens=True))

In [15]:
model2 = SentenceTransformer('all-MiniLM-L6-v2')

#encode the sentences 
truth_embeddings = model2.encode(truth, convert_to_tensor=True)
predicted_embeddings = model2.encode(predicted, convert_to_tensor=True)

scores = []
#compute the similarity scores
for i in range(len(truth_embeddings)):
  cosine_scores = (util.cos_sim(truth_embeddings[i], predicted_embeddings[i]))
  scores.append(cosine_scores)

#compute/find the highest similarity scores
pairs = []
for i in range(len(scores)):
      pairs.append({'index': [i], 'score': scores[i]})


In [17]:
output_df = pd.DataFrame({'truth': truth, 'predicted': predicted, 'score': scores})
output_df['score'] = output_df['score'].apply(lambda x: x.item())
output_df.sort_values(by=['score'], ascending=False, inplace=True)

In [18]:
pd.set_option('display.max_colwidth', None)
output_df

,truth,predicted,score
9,glares thro' the gloom and pours upon my breast,glares thro' all the joys of lifeand now weep'd the,0.600873
5,these sighs to murmur and these tears to flow,these sighs and pangs of grief and joyfellow of the common,0.575053
15,flame thro' my blood and steal me from my urn,flame thro' the grove of their headand charm'd every thought to,0.574694
11,and drags me back to misery and love,and drags me on the dusty road'ring mountain paths and amid the,0.540272
3,these earthborn visions saddening o'er my cell,these earthborn god are'dwith thy glowing and lightning hand they shall,0.491722
1,that wings my pulse and shoots from vein to vein,that wings and shall shew brighti on me with such a lustre,0.484887
10,bids heav'n's bright guard from paraclete remove,bids heav'n's softest motionhis roaring oars may calm,0.470400
6,'tis she 'tis eloisa's form restor'd,'tis she the true queen of truth and a poetess with a thousand,0.423425
12,enjoy thy triumphs dear illusion see,enjoy thy fruits the bestow'dof the birds and the trees for,0.328502
13,this sad apostate from his god to thee,this sad fate thro' my heart to that just landwhere the pain which,0.297925
